TEST Welcha

Test Welcha to wariant Anovy, który jest stosowany, gdy założenie o jednorodności wariancji nie jest spelnione.
Wszystkie inne załozenia zwyklej Anovy i testu Welcha sa takie same i należy je sprawdzić przed wykonaniem analizy.

welch_anova(data=None, dv=None, between=None)
One-way Welch ANOVA.

Parameters
data datapandas.DataFrame Ramka danych

dv= Nazwa kolumny zawierającej zmienną zależną.




dvstring Nazwa kolumny zawierającej zmienną zależną.

betweenstring Nazwa kolumny zawierającej współczynnik między.

Wynik funkcji:
ANOVA summary:

'Source': Factor names

'SS': Sums of squares

'DF': Degrees of freedom

'MS': Mean squares

'F': F-values

'p-unc': uncorrected p-values

'np2': Partial eta-squared


https://pingouin-stats.org/generated/pingouin.welch_anova.html

In [1]:
y1 = [3.79, 5.07, 3.02, 3.22, 3.08, 4.05, 3.13, 2.82]
y2 = [3.12, 2.51, 3.85, 2.56, 4.49, 3.20, 3.41, 3.99]
y3 = [4.76, 3.88, 6.84, 4.15, 2.78, 5.39, 6.41, 2.12]


acznijmy od sprawdzenia założen.
1. Mierzalnosc danych.

Odp: Uznajemy, ze dane sa mierzalne. NIe znamy ich pochodzenia

2. Niezależność zmiennych losowych w rozważanych populacjach (grupach).

Odp: Podobnie jak wyzej. Mozemy uznac, ze zmienne sa niezalezne

3. Normalnosc rozkladu danych.

H0: dane podlegaja rozkładowi normalnemu
H1: dane nie podlegaja rozkładowi normalnemu
     
         
Odp: na poziomie ostotności 0,05 nie mamy podstaw do odrzucenia hipotezy zerowej

4. Brak jednorodnosci wariancji (uwaga na odpowiednie sformulowanie hipotez)

H0: wariancje są takie same
H1: wariancje nie są takie same

odp. Na poziomime istotności 0,05 odrzucamy hiotezę zerową i przyjmujemy hipotezę alternatywną, że próby nie mają jednakowej wariancji.

In [2]:
from scipy.stats import shapiro
data = y1
stat, p =shapiro(data)
print(['Shapiro-Wilk Test: stat=%.3f, p=%3f' %(stat,p)]) 
if p>0.05: print('normal\n') 
else: print('not normal\n')

data = y2
stat, p =shapiro(data)
print(['Shapiro-Wilk Test: stat=%.3f, p=%3f' %(stat,p)]) 
if p>0.05: print('normal\n') 
else: print('not normal\n')

data = y3
stat, p =shapiro(data)
print(['Shapiro-Wilk Test: stat=%.3f, p=%3f' %(stat,p)]) 
if p>0.05: print('normal\n') 
else: print('not normal\n')

import scipy
from scipy.stats import bartlett
scipy.stats.bartlett(y1, y2, y3)

['Shapiro-Wilk Test: stat=0.839, p=0.074047']
normal

['Shapiro-Wilk Test: stat=0.955, p=0.763858']
normal

['Shapiro-Wilk Test: stat=0.969, p=0.893356']
normal



BartlettResult(statistic=6.5399391678445475, pvalue=0.03800758309919509)

Odp:

Jezeli wszystkie założenia sa spelnione, możemy przejść do wykonania testu Welcha.

H0: średnie we wszystkich grupach sa takie same.
H1: średnie we wszystkich grupach nie są takie same.

Jakie sa wnioski z przeprowdzonego testu? Czy patrząc na wartosci srednie wyliczone z prob podjelibysmy taka sama decyzje co do ich rownosci?
Odp:

In [3]:
#import pingouin as pg
import pandas as pd
import numpy as np

#create DataFrame

df1 = pd.DataFrame({'score': [3.79, 5.07, 3.02, 3.22, 3.08, 4.05, 3.13, 2.82],
                   'group': np.repeat(['a'], repeats=8)})
df2 = pd.DataFrame({'score': [3.12, 2.51, 3.85, 2.56, 4.49, 3.20, 3.41, 3.99],
                   'group': np.repeat(['b'], repeats=8)})
df3 = pd.DataFrame({'score': [4.76, 3.88, 6.84, 4.15, 2.78, 5.39, 6.41, 2.12],
                   'group': np.repeat(['c'], repeats=8)})
df= df1.append(df2, ignore_index=True)
df= df.append(df3, ignore_index=True)

C:\Users\jakub\AppData\Local\Temp\ipykernel_6988\2083493433.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df= df1.append(df2, ignore_index=True)
C:\Users\jakub\AppData\Local\Temp\ipykernel_6988\2083493433.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df= df.append(df3, ignore_index=True)


In [4]:
import pingouin as pg
import pandas as pd
import numpy as np

pg.welch_anova(dv='score', between='group', data=df)

,Source,ddof1,ddof2,F,p-unc,np2
0,group,2,13.078353,1.575632,0.243735,0.193737


Całkowita wartość p (0,243735) z tabeli ANOVA jest większa niż α = 0,05, co oznacza, że nie ma podstaw do odrzucenia H0

Dodatek

Następnie możemy wykonać test post hoc Games-Howell, aby dokładnie określić, które średnie grupowe są różne: 

In [5]:
pg.pairwise_gameshowell(dv='score', between='group', data=df)

,A,B,mean(A),mean(B),diff,se,T,df,pval,hedges
0,a,b,3.52250,3.39125,0.13125,0.360710,0.363866,13.912552,0.929980,0.172009
1,a,c,3.52250,4.54125,-1.01875,0.641308,-1.588551,9.764953,0.295553,-0.750951
2,b,c,3.39125,4.54125,-1.15000,0.633214,-1.816131,9.385141,0.216702,-0.858535


Z wartości p widzimy, że średnie  między grupami a i b sa stosunkowo małe w porównaniu do różnic pomiędzy grupami a, c i c,b